<a href="https://colab.research.google.com/github/Oyekanmi/Machine-Learning-New/blob/master/Customer_Churn_ANN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**About the Dataset**

The dataset has the demographic information of bank customers that churns or stays within 6 months.The exited column has either 1 or 0 where 1 means -Customer leaves and 0--means customer stays . How can we predict whether future customer stays within 6 months of enrolment using the dataset ?


In [0]:
#import libraries
import pandas as pd
import numpy as np
import tensorflow as tf

In [0]:
customer_info = pd.read_csv('/Churn_Modelling.csv')  #load customer information

In [25]:
customer_info.head() #check for the first five rows of the data

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [0]:
#Split the data into independet and dependent veriables 
#Since column o to 2 is not necessary , we exclude the columns 

X = customer_info.iloc[:,3:-1]
y= customer_info.iloc[:,-1]

In [27]:
X.shape  #Confirm the dimension of the independent variables

(10000, 10)

In [28]:
y.shape  #Confirm the dimension of the dependent variables

(10000,)

In [29]:
X.head()  #view the first five rows

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,619,France,Female,42,2,0.00,1,1,1,101348.88
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58
2,502,France,Female,42,8,159660.80,3,1,0,113931.57
3,699,France,Female,39,1,0.00,2,0,0,93826.63
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10


**Split into training and testing**

In [0]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train,y_test = train_test_split( X,y,test_size = 0.2 , random_state=42)

**Data Preprocessing**

In [50]:
# Build a pipeline of data preprocessing 
# Check for missing values in the training set
X_train.isna().sum()

CreditScore        0
Geography          0
Gender             0
Age                0
Tenure             0
Balance            0
NumOfProducts      0
HasCrCard          0
IsActiveMember     0
EstimatedSalary    0
dtype: int64

In [0]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import make_column_transformer
from sklearn.preprocessing import StandardScaler

In [0]:
ohe = OneHotEncoder(sparse= False)  # Convert categorical to dummy variables using OheHotEncoder

sc= StandardScaler()    # Perform feature scaling on some features

ct = make_column_transformer((ohe,['Geography','Gender']), (sc,[ 'CreditScore','Age','Tenure','Balance', 'NumOfProducts', 'EstimatedSalary'] ), remainder= 'passthrough' )

In [0]:
X_train = ct.fit_transform(X_train)   #fit the column transformers into the dataset

**Build an ANN**

In [0]:
ann = tf.keras.Sequential() #intialize the ANN

In [0]:
ann.add(tf.keras.layers.Dense(units=6, activation='relu')) # add the input and the first hidden layer

In [0]:
ann.add(tf.keras.layers.Dense(units=6, activation='relu')) # add second hidden layer

In [0]:
ann.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))  #add the final output later

In [0]:
ann.compile(optimizer='adam' , loss= 'binary_crossentropy', metrics=['accuracy'])      #compile the NN

**Train the ANN**

In [61]:
ann.fit(X_train,y_train, batch_size= 32 , epochs=100)     #fit the ANN to the training data

Epoch 1/100
250/250 [==============================] - 0s 1ms/step - loss: 0.5475 - accuracy: 0.7944
Epoch 2/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4628 - accuracy: 0.7945
Epoch 3/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4377 - accuracy: 0.7945
Epoch 4/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4280 - accuracy: 0.7945
Epoch 5/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4211 - accuracy: 0.7945
Epoch 6/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4143 - accuracy: 0.8037
Epoch 7/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4061 - accuracy: 0.8216
Epoch 8/100
250/250 [==============================] - 0s 1ms/step - loss: 0.3976 - accuracy: 0.8290
Epoch 9/100
250/250 [==============================] - 0s 1ms/step - loss: 0.3892 - accuracy: 0.8338
Epoch 10/100
250/250 [==============================] - 0s 1ms/step - loss: 0.3818 - accura

The results shows that the accuracy after 100 epochs is 0.86

**Predictions on unseen dataset**

In [0]:
#Tranform X_test
X_test = ct.transform(X_test)

In [0]:
y_pred = ann.predict(X_test)   

In [0]:
y_pred = (y_pred > 0.5)   # if y_pred > 0.5 , customer leaves , otherwise customer stays

In [65]:
y_pred[:10]

array([[False],
       [False],
       [False],
       [False],
       [False],
       [False],
       [False],
       [False],
       [False],
       [ True]])

In [68]:
#Compute the confusion matrix , accuracy score
from sklearn.metrics import confusion_matrix, accuracy_score
cm= confusion_matrix(y_test, y_pred)   #confusion matrix
print(cm)

print('\n')
accuracy_score(y_test, y_pred)   #calculate the accuracy score

[[1532   75]
 [ 202  191]]




0.8615